In [1]:
import ccxt.async_support as ccxt
import pandas as pd
import asyncio
import os
import sys
import numpy as np
import datetime
from pathlib import Path
from pprint import pprint

In [2]:
p = Path(os.path.abspath(os.getcwd())) / "data" / "limit_orderbook_data"
p = p.resolve()
orderbook_paths = [x for x in p.iterdir() if x.is_file()]
orderbook_paths

[WindowsPath('C:/Users/Cooper/Documents/GitHub/DeepLOB_Trader/src/data/limit_orderbook_data/binanceus_ADA-USDT_1621823814689_1621867014689_lob.pkl'),
 WindowsPath('C:/Users/Cooper/Documents/GitHub/DeepLOB_Trader/src/data/limit_orderbook_data/binanceus_DOGE-USDT_1620688812541_1620688872541_lob.pkl'),
 WindowsPath('C:/Users/Cooper/Documents/GitHub/DeepLOB_Trader/src/data/limit_orderbook_data/binance_ADA-USDT_1621823814689_1621867014689_lob.pkl'),
 WindowsPath('C:/Users/Cooper/Documents/GitHub/DeepLOB_Trader/src/data/limit_orderbook_data/kraken_ADA-USDT_1621823814689_1621867014689_lob.pkl'),
 WindowsPath('C:/Users/Cooper/Documents/GitHub/DeepLOB_Trader/src/data/limit_orderbook_data/kraken_DOGE-USDT_1620688812541_1620688872541_lob.pkl'),
 WindowsPath('C:/Users/Cooper/Documents/GitHub/DeepLOB_Trader/src/data/limit_orderbook_data/kraken_DOGE-USDT_1620689598600_1620732798600_lob.pkl'),
 WindowsPath('C:/Users/Cooper/Documents/GitHub/DeepLOB_Trader/src/data/limit_orderbook_data/kucoin_DOGE-USDT

In [189]:
df = pd.read_pickle(orderbook_paths[3])

In [190]:
df.head(3)

,date,exchange_id,symbol,asks,bids
0,2021-05-24T02:37:00.792Z,kraken,ADA/USDT,"[[1.333264, 981.791, 1621823819], [1.333265, 3...","[[1.332844, 9.846, 1621823416], [1.332482, 12...."
1,2021-05-24T02:37:03.809Z,kraken,ADA/USDT,"[[1.333264, 981.791, 1621823819], [1.333265, 3...","[[1.332844, 9.846, 1621823416], [1.332482, 12...."
2,2021-05-24T02:37:06.805Z,kraken,ADA/USDT,"[[1.332481, 982.352, 1621823823], [1.332482, 1...","[[1.329828, 5082.003, 1621823821], [1.329403, ..."


In [191]:
len(df['asks'][0])

40

In [192]:
len(df['bids'][0])

40

In [193]:
df['bids'][0][:3]

[[1.332844, 9.846, 1621823416],
 [1.332482, 12.673, 1621823414],
 [1.329816, 981.824, 1621823818]]

In [194]:
df['asks'][0][:3]

[[1.333264, 981.791, 1621823819],
 [1.333265, 3750.772, 1621823818],
 [1.333419, 3750.772, 1621823818]]

In [195]:
def calculate_midprice(row):
    return (row['bids'][0][0] + row['asks'][0][0]) / 2.0

In [196]:
df['midprice'] = df.apply(calculate_midprice, axis=1)

In [197]:
#df['date'] = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S")

In [198]:
#df['time_delta'] = df.date.diff()
#df = df.fillna(df['time_delta'])

In [199]:
df.drop(['date', 'exchange_id', 'symbol'], axis=1, inplace=True)

In [200]:
df.head(3)

,asks,bids,midprice
0,"[[1.333264, 981.791, 1621823819], [1.333265, 3...","[[1.332844, 9.846, 1621823416], [1.332482, 12....",1.333054
1,"[[1.333264, 981.791, 1621823819], [1.333265, 3...","[[1.332844, 9.846, 1621823416], [1.332482, 12....",1.333054
2,"[[1.332481, 982.352, 1621823823], [1.332482, 1...","[[1.329828, 5082.003, 1621823821], [1.329403, ...",1.331155


In [201]:
df['midprice'].describe()

count    14461.000000
mean         1.430330
std          0.055083
min          1.309484
25%          1.370947
50%          1.449089
75%          1.473569
max          1.547125
Name: midprice, dtype: float64

In [202]:
#df['time_delta'].describe()

In [203]:
df['midprice_z'] = (df['midprice'] - df['midprice'].mean()) / df['midprice'].std(ddof=0)

In [204]:
df

,asks,bids,midprice,midprice_z
0,"[[1.333264, 981.791, 1621823819], [1.333265, 3...","[[1.332844, 9.846, 1621823416], [1.332482, 12....",1.333054,-1.766060
1,"[[1.333264, 981.791, 1621823819], [1.333265, 3...","[[1.332844, 9.846, 1621823416], [1.332482, 12....",1.333054,-1.766060
2,"[[1.332481, 982.352, 1621823823], [1.332482, 1...","[[1.329828, 5082.003, 1621823821], [1.329403, ...",1.331155,-1.800546
3,"[[1.332481, 982.352, 1621823823], [1.332482, 1...","[[1.329828, 5082.003, 1621823821], [1.329403, ...",1.331155,-1.800546
4,"[[1.33362, 981.057, 1621823828], [1.333746, 37...","[[1.332251, 1296.0, 1621823828], [1.331336, 95...",1.332936,-1.768212
...,...,...,...,...
14456,"[[1.47252, 2332.575, 1621867339], [1.473078, 3...","[[1.469109, 4185.355, 1621867338], [1.46852, 5...",1.470814,0.734997
14457,"[[1.47252, 2332.575, 1621867339], [1.474084, 3...","[[1.470009, 4226.609, 1621867340], [1.469996, ...",1.471264,0.743167
14458,"[[1.475776, 5000.0, 1621867347], [1.475788, 32...","[[1.472183, 4232.991, 1621867343], [1.472182, ...",1.473979,0.792458
14459,"[[1.476814, 6.825, 1621867349], [1.476815, 44....","[[1.473421, 4102.871, 1621867350], [1.473418, ...",1.475118,0.813118


In [205]:
df['midprice_z'].describe()

count    1.446100e+04
mean    -5.031435e-16
std      1.000035e+00
min     -2.193976e+00
25%     -1.078109e+00
50%      3.405772e-01
75%      7.850143e-01
max      2.120423e+00
Name: midprice_z, dtype: float64

In [206]:
df

,asks,bids,midprice,midprice_z
0,"[[1.333264, 981.791, 1621823819], [1.333265, 3...","[[1.332844, 9.846, 1621823416], [1.332482, 12....",1.333054,-1.766060
1,"[[1.333264, 981.791, 1621823819], [1.333265, 3...","[[1.332844, 9.846, 1621823416], [1.332482, 12....",1.333054,-1.766060
2,"[[1.332481, 982.352, 1621823823], [1.332482, 1...","[[1.329828, 5082.003, 1621823821], [1.329403, ...",1.331155,-1.800546
3,"[[1.332481, 982.352, 1621823823], [1.332482, 1...","[[1.329828, 5082.003, 1621823821], [1.329403, ...",1.331155,-1.800546
4,"[[1.33362, 981.057, 1621823828], [1.333746, 37...","[[1.332251, 1296.0, 1621823828], [1.331336, 95...",1.332936,-1.768212
...,...,...,...,...
14456,"[[1.47252, 2332.575, 1621867339], [1.473078, 3...","[[1.469109, 4185.355, 1621867338], [1.46852, 5...",1.470814,0.734997
14457,"[[1.47252, 2332.575, 1621867339], [1.474084, 3...","[[1.470009, 4226.609, 1621867340], [1.469996, ...",1.471264,0.743167
14458,"[[1.475776, 5000.0, 1621867347], [1.475788, 32...","[[1.472183, 4232.991, 1621867343], [1.472182, ...",1.473979,0.792458
14459,"[[1.476814, 6.825, 1621867349], [1.476815, 44....","[[1.473421, 4102.871, 1621867350], [1.473418, ...",1.475118,0.813118


In [277]:
df['bids']

[1.332482, 12.673, 1621823414]

In [227]:
np_bids = np.stack(df['bids'])
np_asks = np.stack(df['asks'])

In [228]:
np_bids.shape

(14461, 40, 3)

In [250]:
np_bids_no_time = np_bids[:,:,0:2]
np_asks_no_time = np_asks[:,:,0:2]

In [251]:
np_bids_no_time.shape

(14461, 40, 2)

In [264]:
np_bids_no_time

array([[[1.332844e+00, 9.846000e+00],
        [1.332482e+00, 1.267300e+01],
        [1.329816e+00, 9.818240e+02],
        ...,
        [1.300909e+00, 1.512130e+02],
        [1.298045e+00, 1.078860e+02],
        [1.294713e+00, 1.292100e+01]],

       [[1.332844e+00, 9.846000e+00],
        [1.332482e+00, 1.267300e+01],
        [1.329816e+00, 9.818240e+02],
        ...,
        [1.300909e+00, 1.512130e+02],
        [1.298045e+00, 1.078860e+02],
        [1.294713e+00, 1.292100e+01]],

       [[1.329828e+00, 5.082003e+03],
        [1.329403e+00, 7.769910e+02],
        [1.329250e+00, 9.824110e+02],
        ...,
        [1.294637e+00, 1.936400e+04],
        [1.294339e+00, 1.013900e+01],
        [1.292565e+00, 9.693000e+03]],

       ...,

       [[1.472183e+00, 4.232991e+03],
        [1.472182e+00, 3.387482e+03],
        [1.471594e+00, 1.006567e+03],
        ...,
        [1.437158e+00, 1.341296e+03],
        [1.436204e+00, 1.292100e+01],
        [1.435946e+00, 7.323000e+00]],

       [[1.4734

In [292]:
np_bids_asks_stacked = np.stack((np_bids_no_time, np_asks_no_time), axis=1)

In [293]:
# [rows, bids or asks, 40 levels, price-amount]
np_bids_asks_stacked.shape

(14461, 2, 40, 2)

In [298]:
# asks
np_bids_asks_stacked[0, 1, :]

array([[1.3332640e+00, 9.8179100e+02],
       [1.3332650e+00, 3.7507720e+03],
       [1.3334190e+00, 3.7507720e+03],
       [1.3337570e+00, 3.6004030e+03],
       [1.3337860e+00, 4.8747330e+03],
       [1.3339180e+00, 1.1104510e+03],
       [1.3340630e+00, 2.5131040e+03],
       [1.3343380e+00, 1.1320230e+03],
       [1.3345940e+00, 9.0890000e+02],
       [1.3346260e+00, 7.7669600e+02],
       [1.3348070e+00, 1.5534640e+03],
       [1.3373600e+00, 9.6000000e+00],
       [1.3373610e+00, 7.9520000e+02],
       [1.3394250e+00, 3.7766000e+01],
       [1.3530610e+00, 5.5214280e+03],
       [1.3530620e+00, 3.9740000e+03],
       [1.3530630e+00, 1.1509000e+01],
       [1.3544950e+00, 7.3921300e+03],
       [1.3544990e+00, 9.5460000e+03],
       [1.3545000e+00, 1.2046174e+04],
       [1.3600000e+00, 7.3685000e+02],
       [1.3602230e+00, 1.0441400e+02],
       [1.3630380e+00, 1.4385580e+03],
       [1.3630940e+00, 5.3297000e+01],
       [1.3664050e+00, 4.0282000e+01],
       [1.3675500e+00, 4.

In [300]:
# bids
np_bids_asks_stacked[0, 0, :]

array([[1.3328440e+00, 9.8460000e+00],
       [1.3324820e+00, 1.2673000e+01],
       [1.3298160e+00, 9.8182400e+02],
       [1.3298130e+00, 3.7477740e+03],
       [1.3298010e+00, 7.7714600e+02],
       [1.3296480e+00, 5.4890800e+02],
       [1.3296470e+00, 1.5542910e+03],
       [1.3296020e+00, 1.7010000e+03],
       [1.3295810e+00, 1.1193760e+03],
       [1.3290950e+00, 3.7494770e+03],
       [1.3289530e+00, 1.1130550e+03],
       [1.3285560e+00, 9.2080000e+02],
       [1.3282070e+00, 2.7967500e+03],
       [1.3274990e+00, 1.9572870e+03],
       [1.3273570e+00, 9.8800000e+02],
       [1.3266590e+00, 2.9860000e+03],
       [1.3266580e+00, 1.0555900e+02],
       [1.3265350e+00, 9.5460000e+03],
       [1.3240440e+00, 3.7764000e+01],
       [1.3238500e+00, 1.4859300e+02],
       [1.3226840e+00, 7.4696940e+03],
       [1.3213610e+00, 1.4588400e+03],
       [1.3203320e+00, 1.1632000e+01],
       [1.3173520e+00, 1.2818000e+01],
       [1.3170130e+00, 7.4467120e+03],
       [1.3156960e+00, 6.

In [325]:
def convert_bids_asks_to_zscores(df):
    
    bid_price_zscore = df[:, 0, :, 0]
    print(len(bid_price_zscore[0]))
    
    return bid_price_zscore

In [326]:
convert_bids_asks_to_zscores(np_bids_asks_stacked)

40


array([[1.332844, 1.332482, 1.329816, ..., 1.300909, 1.298045, 1.294713],
       [1.332844, 1.332482, 1.329816, ..., 1.300909, 1.298045, 1.294713],
       [1.329828, 1.329403, 1.32925 , ..., 1.294637, 1.294339, 1.292565],
       ...,
       [1.472183, 1.472182, 1.471594, ..., 1.437158, 1.436204, 1.435946],
       [1.473421, 1.473418, 1.473388, ..., 1.448274, 1.444769, 1.444767],
       [1.474983, 1.474982, 1.47497 , ..., 1.45057 , 1.449119, 1.448274]])

In [254]:
np_bid_price_zscore = ( np_bids_no_time[:,:,0] - np_bids_no_time[:,:,0].mean() ) / np_bids_no_time[:,:,0].std()

In [322]:
np_bid_price_zscore

array([[-1.48683507, -1.49326345, -1.54060622, ..., -2.05393608,
        -2.10479493, -2.16396451],
       [-1.48683507, -1.49326345, -1.54060622, ..., -2.05393608,
        -2.10479493, -2.16396451],
       [-1.54039313, -1.54794027, -1.55065724, ..., -2.16531411,
        -2.17060599, -2.20210864],
       ...,
       [ 0.98754377,  0.98752601,  0.97708432, ...,  0.36557061,
         0.3486295 ,  0.34404794],
       [ 1.00952815,  1.00947487,  1.00894213, ...,  0.56296828,
         0.50072657,  0.50069106],
       [ 1.03726611,  1.03724835,  1.03703525, ...,  0.6037406 ,
         0.57797378,  0.56296828]])

In [256]:
np_bid_amount_zscore = ( np_bids_no_time[:,:,1] - np_bids_no_time[:,:,1].mean() ) / np_bids_no_time[:,:,1].std()

In [257]:
np_bid_amount_zscore

array([[-0.40831195, -0.40809507, -0.33374665, ..., -0.39746698,
        -0.40079081, -0.40807605],
       [-0.40831195, -0.40809507, -0.33374665, ..., -0.39746698,
        -0.40079081, -0.40807605],
       [-0.01920141, -0.34946042, -0.33370162, ...,  1.07644202,
        -0.40828947,  0.33453124],
       ...,
       [-0.08433337, -0.14919659, -0.33184849, ..., -0.30616976,
        -0.40807605, -0.4085055 ],
       [-0.09431552, -0.34125429, -0.33013875, ..., -0.39593513,
        -0.37197335, -0.398622  ],
       [-0.09077866, -0.02549227, -0.14968151, ...,  0.63043404,
         2.58404525, -0.39593513]])

In [259]:
np_zscore_bids = np.stack((np_bid_price_zscore, np_bid_amount_zscore), axis=2)

In [260]:
np_zscore_bids

array([[[-1.48683507, -0.40831195],
        [-1.49326345, -0.40809507],
        [-1.54060622, -0.33374665],
        ...,
        [-2.05393608, -0.39746698],
        [-2.10479493, -0.40079081],
        [-2.16396451, -0.40807605]],

       [[-1.48683507, -0.40831195],
        [-1.49326345, -0.40809507],
        [-1.54060622, -0.33374665],
        ...,
        [-2.05393608, -0.39746698],
        [-2.10479493, -0.40079081],
        [-2.16396451, -0.40807605]],

       [[-1.54039313, -0.01920141],
        [-1.54794027, -0.34946042],
        [-1.55065724, -0.33370162],
        ...,
        [-2.16531411,  1.07644202],
        [-2.17060599, -0.40828947],
        [-2.20210864,  0.33453124]],

       ...,

       [[ 0.98754377, -0.08433337],
        [ 0.98752601, -0.14919659],
        [ 0.97708432, -0.33184849],
        ...,
        [ 0.36557061, -0.30616976],
        [ 0.3486295 , -0.40807605],
        [ 0.34404794, -0.4085055 ]],

       [[ 1.00952815, -0.09431552],
        [ 1.00947487, -0.34